In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)

Using PyTorch version: 1.12.1+cu113  Device: cuda


In [3]:
batch_size = 32
epochs = 10

In [ ]:
# data Augmentation이 적용된 데이터 로더
train_dataset = datasets.CIFAR10(root = '../data/CIFAR_10', train=True, download = True
                 , transform=transforms.Compose([
                     transforms.RandomHorizontalFlip(),
                     transforms.ToTensor(),
                     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
                 ])
                 )
test_dataset = datasets.CIFAR10(root = '../data/CIFAR_10', train=False, download = True
                 , transform=transforms.Compose([
                     transforms.RandomHorizontalFlip(),
                     transforms.ToTensor(),
                     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
                 ])
                 )
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,batch_size = batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset = train_dataset,batch_size = batch_size,shuffle=False)

In [ ]:
from torchvision.models.mobilenetv3 import Weights
# 파이토치에서 제공하는 ResNet34모델을 불러온후 FC 층 추가 및 output크기 설정
import torchvision.models as models
model = models.resnet34(weights = False )

In [14]:
num_ftrs =  model.fc.in_features

In [15]:
model.fc = nn.Linear(num_ftrs,10)
model = model.to(DEVICE)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.CrossEntropyLoss()
model

In [17]:
def train(model, train_loader, optimizer, log_interval):
  model.train()
  for batch_idx, (image,label) in enumerate(train_loader):
    image = image.to(DEVICE)
    label = label.to(DEVICE)
    optimizer.zero_grad()
    output = model(image)
    loss = criterion(output, label)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
          epochs, batch_idx * len(image), 
          len(train_loader.dataset), 100. * batch_idx / len(train_loader), 
          loss.item()))


In [18]:
# 검증용 함수 정의
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [19]:
# ResNet34 학습 train, test loss accuracy 확인하기
for epoch in range(1, epochs+1):
  train(model, train_loader, optimizer,200)
  test_loss, test_accuracy =  evaluate(model, test_loader)
  print(f"[epoch:{epoch}], loss : {test_loss}  accuracy : {test_accuracy}")

Train Epoch: 10 [0/50000 (0%)]	Train Loss: 2.571666
Train Epoch: 10 [6400/50000 (13%)]	Train Loss: 2.070665
Train Epoch: 10 [12800/50000 (26%)]	Train Loss: 1.592525
Train Epoch: 10 [19200/50000 (38%)]	Train Loss: 1.466715
Train Epoch: 10 [25600/50000 (51%)]	Train Loss: 1.223977
Train Epoch: 10 [32000/50000 (64%)]	Train Loss: 1.330877
Train Epoch: 10 [38400/50000 (77%)]	Train Loss: 1.239332
Train Epoch: 10 [44800/50000 (90%)]	Train Loss: 1.341710
[epoch:1], loss : 0.034280876103639606  accuracy : 61.326
Train Epoch: 10 [0/50000 (0%)]	Train Loss: 1.063032
Train Epoch: 10 [6400/50000 (13%)]	Train Loss: 0.988820
Train Epoch: 10 [12800/50000 (26%)]	Train Loss: 1.211275
Train Epoch: 10 [19200/50000 (38%)]	Train Loss: 1.199988
Train Epoch: 10 [25600/50000 (51%)]	Train Loss: 1.401958
Train Epoch: 10 [32000/50000 (64%)]	Train Loss: 1.087926
Train Epoch: 10 [38400/50000 (77%)]	Train Loss: 1.341031
Train Epoch: 10 [44800/50000 (90%)]	Train Loss: 1.169402
[epoch:2], loss : 0.030974212233424187  ac

In [ ]:
# 미세조정 fineTurning
model_ft = models.resnet34(weights = True )
num_ftrs_finturning = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs_finturning,10)
model_ft = model_ft.to(DEVICE)
optimizer = torch.optim.Adam(model_ft.parameters(), lr = 0.001)
for epoch in range(1, epochs+1):
  train(model, train_loader, optimizer,200)
  test_loss, test_accuracy =  evaluate(model, test_loader)
  print(f"[epoch:{epoch}], loss : {test_loss}  accuracy : {test_accuracy}")

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

Train Epoch: 10 [0/50000 (0%)]	Train Loss: 0.257738
Train Epoch: 10 [6400/50000 (13%)]	Train Loss: 0.356691
Train Epoch: 10 [12800/50000 (26%)]	Train Loss: 0.348255
Train Epoch: 10 [19200/50000 (38%)]	Train Loss: 0.455195
Train Epoch: 10 [25600/50000 (51%)]	Train Loss: 0.484703
Train Epoch: 10 [32000/50000 (64%)]	Train Loss: 0.344703
Train Epoch: 10 [38400/50000 (77%)]	Train Loss: 0.301215
Train Epoch: 10 [44800/50000 (90%)]	Train Loss: 0.380047
[epoch:1], loss : 0.01115169466048479  accuracy : 87.852
Train Epoch: 10 [0/50000 (0%)]	Train Loss: 0.449837
Train Epoch: 10 [6400/50000 (13%)]	Train Loss: 0.728127
Train Epoch: 10 [12800/50000 (26%)]	Train Loss: 0.304030
Train Epoch: 10 [19200/50000 (38%)]	Train Loss: 0.352457
Train Epoch: 10 [25600/50000 (51%)]	Train Loss: 0.534126
Train Epoch: 10 [32000/50000 (64%)]	Train Loss: 0.400349
Train Epoch: 10 [38400/50000 (77%)]	Train Loss: 0.261137
Train Epoch: 10 [44800/50000 (90%)]	Train Loss: 0.808286
[epoch:2], loss : 0.011022283936440945  acc